# Chapter 02: Use of E-Utilities

Ray LeClair \<2024-07-06 Sat\>

## Objectives

Since the OntoGPT `pubmed-annotate` function requires a PubMed
Identifier (PMID) as an argument, the objective of this document
involves using the E-Utilities to search PubMed for a given title.

See:

-   [Chapter-04-ontoGPT.ipynb](Chapter-04-OntoGPT.ipynb)
-   [E-Utilities](https://www.ncbi.nlm.nih.gov/books/NBK25499/)

### Background

The Entrez Programming Utilities (E-utilities) provide a stable API
into the Entrez query and database system at the National Center for
Biotechnology Information (NCBI).

The following sections describe various development environments

See: [springbok-nlm-kn/README.md](https://github.com/ralatsdc/springbok-nlm-kn/blob/main/README.md)

### Jupyter Notebook development environment

Launch Jupyter Notebook from a terminal in which `.zshenv` has been
sourced, and the virtual environment has been activated.

### Emacs Org Mode development environment

Launch Emacs from a terminal in which `.zshenv` has been sourced, then
evaluate this code block to activate the virtual environment:

``` commonlisp
(pyvenv-activate "../../.venv")
```

## Identification of a title within PubMed

To begin, we import modules, and assign module scope variables:

In [ ]:
import logging
import os
from time import sleep
from traceback import print_exc
from urllib import parse

from bs4 import BeautifulSoup
import requests

DATA_DIR = "../data"

EUTILS_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
NCBI_EMAIL = os.environ.get("NCBI_EMAIL")
NCBI_API_KEY = os.environ.get("NCBI_API_KEY")
NCBI_API_SLEEP = 1
PUBMED = "pubmed"


Now consider, for example, the citations provided by CELLxGENE for the
human lung cell datasets. We looked at how to get the datasets, and
the titles of the corresponding publications earlier.

See: [Chapter-01-CELLxGENE.ipynb](Chapter-01-CELLxGENE.ipynb)

In [ ]:
from CELLxGENE import get_metadata_and_datasets, get_title
try:
    datasets, counts, var, obs = get_metadata_and_datasets()
    citation = datasets["citation"].iloc[4]
    title = get_title(citation)
except Exception:
    print_exc()


Now we can write a function using the E-Utilities to search PubMed for
the title:

In [ ]:
def get_pmid_for_title(title):
    """Search PubMed using a title to find the corresponding PMID.

    Parameters
    ----------
    title : str
       The title to use in the search

    Returns
    -------
    pmid : str
       The PubMed identifier found
    """
    # Need a default return value
    pmid = None

    # Search PubMed
    if title is None:
        return pmid
    print(f"Getting PMID for title: '{title}'")
    search_url = EUTILS_URL + "esearch.fcgi"
    print(search_url)
    params = {
        "db": PUBMED,
        "term": title,
        "field": "title",
        "retmode": "json",
        # "retmax": 0,
        "email": NCBI_EMAIL,
        "api_key": NCBI_API_KEY,
    }
    print(params)
    sleep(NCBI_API_SLEEP)
    response = requests.get(search_url, params=parse.urlencode(params, safe=","))
    if response.status_code == 200:
        data = response.json()
        resultcount = int(data["esearchresult"]["count"])

        if resultcount > 1:
            # Response contains more than once result, so fetch each
            # PMID until title matches
            logging.warning(f"PubMed returned more than one result for title: {title}")
            for _pmid in data["esearchresult"]["idlist"]:
                _title = get_title_for_pmid(_pmid)
                if (
                    _title == title + "."
                ):  # PMID fetch includes period in title, title search does not
                    pmid = _pmid
                    break

        else:
            pmid = data["esearchresult"]["idlist"][0]

        print(f"Found PMID: {pmid} for title: '{title}'")

    elif response.status_code == 429:
        logging.error("Too many requests to NCBI API. Try again later, or use API key.")

    else:
        logging.error(f"Encountered error in searching PubMed: {response.status_code}")

    return pmid


In the process of testing this function we discover that multiple
PubMed Identifiers (PMIDs) can be returned. So we need to write a
function that returns the title for a given PMID to confirm which of
the multiple PMIDs returned is correct as follows:

In [ ]:
def get_title_for_pmid(pmid):
    """Fetch from PubMed using a PMID to find the corresponding title.

    Parameters
    ----------
    pmid : str
       The PubMed identifier to use in the fetch

    Returns
    -------
    title : str
       The title fetched
    """
    # Need a default return value
    title = None

    # Fetch from PubMed
    fetch_url = EUTILS_URL + "efetch.fcgi"
    params = {
        "db": PUBMED,
        "id": pmid,
        "rettype": "xml",
        "email": NCBI_EMAIL,
        "api_key": NCBI_API_KEY,
    }
    sleep(NCBI_API_SLEEP)
    response = requests.get(fetch_url, params=parse.urlencode(params, safe=","))
    if response.status_code == 200:
        xml_data = response.text

        # Got the page, so parse it, and search for the title
        fullsoup = BeautifulSoup(xml_data, "xml")
        found = fullsoup.find("ArticleTitle")
        if found:
            title = found.text

    else:
        logging.error(
            f"Encountered error in fetching from PubMed: {response.status_code}"
        )

    return title


Now we can get the PMID for the title:

In [ ]:
pmid = get_pmid_for_title(title)
print(f"PMID: {pmid} found for title: {title}")


Next, in Chapter 03 we write a function to run NS-Forest on a dataset.

See: [Chapter-03-NS-Forest.ipynb](Chapter-03-NS-Forest.ipynb)